In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import pickle
import numpy as np

In [40]:
all_songs = pd.read_csv('all_songs.csv')
some_songs = all_songs.loc[all_songs['lyrics'].str.startswith('[').fillna(False)]
some_songs

,song title,lyrics,artist
0,In Da Club,"[Intro]\nGo, go, go, go, go, go\nGo Shorty, it...",50 Cent
1,21 Questions,[Ad-Libs]\nNew York City\nYou are now rockin'\...,50 Cent
2,Many Men (Wish Death),"[Skit]\nMan, we gotta go get somethin' to eat\...",50 Cent
3,My Life,"[Chorus]\nMy, yeah, yeah, mmm\nMy life, my lif...",50 Cent
4,Patiently Waiting,"[Intro]\nHey Em, you know you're my favorite w...",50 Cent
...,...,...,...
575,Cruisin’,"[Intro]\n(Crusin')\n\n[Verse 1]\nBaby, let's c...",Smokey Robinson
576,Really Gonna Miss You,[Verse 1]\nReally gonna miss you\nIt's really ...,Smokey Robinson
579,The Agony and the Ecstasy,[Verse 1]\nWhat's it all about this crazy love...,Smokey Robinson
582,Ooh Baby Baby,[Verse 1]\nI did you wrong\nMy heart went out ...,Smokey Robinson


In [ ]:
def split_by_bar(song):
    verses = split_by_verse(song, removeN=False)
    bars = []
    for verse in verses:
        t_bars = verse.split('\n')

In [41]:
def split_by_verse(song, removeN = True):
    verses = song.split('[')
    for ind in range(1, len(verses)):
        verses[ind] = verses[ind].split(']')[1]
        verses[ind] = verses[ind][1:]
        if removeN:
            verses[ind] = re.sub('\n', ' ', verses[ind])
        verses[ind] = re.sub('  ', '', verses[ind])
    return verses[1:]

def get_embeddings(msgs):
    encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')
    embeddings = encoder.encode(msgs)
    return torch.unsqueeze(torch.from_numpy(embeddings), 0)

In [44]:
X = some_songs.lyrics.values
y = pd.get_dummies(some_songs.artist.values).to_numpy()

In [36]:
from Saving import *

In [32]:
X = X[:488:5]
y = y[:488:5]

for ind in tqdm(range(len(X))):
    try:
        verses = split_by_verse(X[ind])
        X[ind] = get_embeddings(verses)
    except IndexError:
        # remove the y entry for failed X
        np.delete(y, ind, 0)
        continue

100%|██████████| 98/98 [03:58<00:00,  2.43s/it]


In [45]:
X = openPk('enc_songs')

In [57]:
np.array(newY).shape

(486, 13)

In [58]:
newX = []
newY = []
for ind in range(X.shape[0]):
    if torch.is_tensor(X[ind]):
        newX.append(X[ind])
        newY.append(y[ind, :])
newY = np.array(newY)

In [59]:
newX

[tensor([[[-0.0065, -0.1977,  0.5451,  ...,  0.1583, -0.4185, -0.3211],
          [-0.4460, -0.3917,  0.3042,  ..., -0.4725,  0.1028, -0.2095],
          [-0.2763, -0.1570,  0.5128,  ...,  0.3289, -0.2283, -0.2563],
          [ 0.0282, -0.2924, -0.2349,  ..., -0.3065,  0.1411, -0.6851],
          [ 0.1757,  0.0794,  0.2891,  ..., -0.0615,  0.1933, -0.3842],
          [-0.1044,  0.0380,  0.4135,  ..., -0.1338, -0.1524, -0.7642]]]),
 tensor([[[-0.2163, -0.4261, -0.0495,  ..., -0.4558, -0.3791, -1.4157],
          [-0.3366,  0.0466,  0.9403,  ...,  0.0525, -0.1953,  0.1601],
          [-0.4842, -0.3846,  0.6890,  ..., -0.2891, -0.4213, -0.9010],
          ...,
          [-0.4557, -0.3342,  0.5295,  ...,  0.3058, -0.0609, -0.2971],
          [-0.4842, -0.3846,  0.6890,  ..., -0.2891, -0.4213, -0.9010],
          [-0.0734, -0.2638,  0.8337,  ...,  0.1134, -0.1690, -0.6548]]]),
 tensor([[[-0.3369, -0.1543,  0.4928,  ..., -0.2273,  0.7324,  0.0172],
          [ 0.3659, -0.4403,  0.8316,  ...,

In [ ]:
from Saving import *
savePk('enc_songs', X) 

In [60]:
x_train, x_test, y_train, y_test = train_test_split(newX, newY, train_size = 0.9, shuffle = True)

In [61]:
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [68]:
class BaseLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(BaseLSTM, self).__init__()
        
        # define hidden size
        self.hidden_size = hidden_size

        # define lstm layer
        self.lstm = nn.LSTM(input_size, hidden_size, 1, batch_first = True, bidirectional=True)

        # define fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        
        # softmax activation
        self.softmax = nn.Softmax(dim=0)

    def forward(self, input):
        
        #init hidden states
        h0 = torch.zeros(2, 1, self.hidden_size)
        c0 = torch.zeros(2, 1, self.hidden_size)
        
        # pass input and h0, c0 into LSTM
        out, (h_out, c_out) = self.lstm(input, (h0, c0))
        
        out = out.reshape(-1, self.hidden_size)
        out = self.fc(out)
        artist_scores = self.softmax(out[-1])
        artist_scores = torch.unsqueeze(artist_scores, 0)
        return artist_scores

In [78]:
# training
# model = BaseLSTM(input_size=768, hidden_size=256, output_size=13)
# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.1)

model.train()
for e in tqdm(range(50)):
    for x, y in zip(x_train, y_train):
        model.zero_grad()

        artist_scores = model(x)

        loss = loss_function(artist_scores, torch.unsqueeze(torch.argmax(y), 0))
        loss.backward()
        optimizer.step()

100%|██████████| 50/50 [10:14<00:00, 12.29s/it]


In [79]:
def Accuracy(xs, ys):
    correct = 0
    for i in range(len(xs)):
        # run through model
        pred = model(xs[i])
        # softmax distribution
        pred = nn.functional.softmax(pred[-1], dim=0).data
        # calc argmax
        pred = torch.argmax(pred).item()
        # sum up correct predictions
        correct += (pred == torch.unsqueeze(torch.argmax(ys[i]), 0))
    return correct.item()/len(xs)

test_acc = Accuracy(x_test, y_test)
print('{}% Test Accuracy'.format(test_acc*100))

10.204081632653061% Test Accuracy


In [80]:
train_acc = Accuracy(x_train, y_train)
print('{}% Train Accuracy'.format(train_acc*100))

10.755148741418765% Train Accuracy


In [74]:
y_train.size()

torch.Size([437, 13])